In [155]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [156]:
df_geo = pd.read_csv('Data - Hack UPC/geo_params.csv')
df_sales = pd.read_csv('Data - Hack UPC/sales.csv')
df_sku = pd.read_csv('Data - Hack UPC/sku.csv')
test = pd.read_csv('Data - Hack UPC/test.csv')

## Geo file

The geo_params variables are the following:
   - **geoCluster**: Number of a concrete zone in a city
   - **cityId**: City in which the geoCluster came from.

In [157]:
print('Number of occurences per city: \n', df_geo.cityId.value_counts())

Number of occurences per city: 
 0     450
1      64
25      1
Name: cityId, dtype: int64


Imbalance dataset in the number of geoClusters, zones registred. As we can see we have the city more splited in the city Id 0, the city Id 1 has less number of geoClusters and city with Id 25 only 1. We can hypothesis for the future the next:

    - The number of geoClusters per city represent how big is a city compared with the others. We have also to think this dataset is not full for the fact we have id 0, 1 and 25 is not in order.

## Sku file

The sku variables are the following:
   - **SKU**: Unique code of SKU (Stock Keeping Unit), which identifies the smallest sales unit.
   - **Category**: The name of the set in which the Unit belongs.
   - **Type**: ?
   - **brandId**: Id of the branding in which this product belongs
   - **largerUnitQuantity**: ?
   - **Units**: Unkown, suppose number of units per SKU but itself express the smallest sales unit.
   - **trademark**: trademark number is the number that is assigned to a pending or registered trademark.
   - **countryOfOrigin**: Origin in which the product comes
   - **Group**: Set in which categories belong to

In [158]:
df_sku.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SKU                60 non-null     int64  
 1   Category           55 non-null     object 
 2   Type               55 non-null     object 
 3   brandId            32 non-null     float64
 4   lagerUnitQuantity  60 non-null     float64
 5   Units              0 non-null      float64
 6   trademark          49 non-null     float64
 7   countryOfOrigin    33 non-null     float64
 8   Group              60 non-null     object 
dtypes: float64(5), int64(1), object(3)
memory usage: 4.3+ KB


## Data change

In [159]:
df_sku.Category.fillna(df_sku.Group, inplace=True)
df_sku.brandId.fillna(0, inplace=True) # Meaning other
df_sku.trademark.fillna(0, inplace=True) # Meaning other
df_sku.countryOfOrigin.fillna(0, inplace=True) # Meaning other

df_sku["Category"] = df_sku["Category"].astype("category")
df_sku["Group"] = df_sku["Group"].astype("category")
df_sku["brandId"] = df_sku["brandId"].astype("Int64")
df_sku["trademark"] = df_sku["trademark"].astype("Int64")
df_sku["countryOfOrigin"] = df_sku["countryOfOrigin"].astype("Int64")
df_sku["lagerUnitQuantity"] = df_sku["lagerUnitQuantity"].astype("Float64")

df_sku.drop(['Units', 'Type'], axis=1, inplace=True)

df_sku.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   SKU                60 non-null     int64   
 1   Category           60 non-null     category
 2   brandId            60 non-null     Int64   
 3   lagerUnitQuantity  60 non-null     Float64 
 4   trademark          60 non-null     Int64   
 5   countryOfOrigin    60 non-null     Int64   
 6   Group              60 non-null     category
dtypes: Float64(1), Int64(3), category(2), int64(1)
memory usage: 4.4 KB


## Sales file

The sku variables are the following:
   - **ID**: id of the sale
   - **geoCluster**: Key of geo csv
   - **SKU**: Key of SKU csv
   - **date**: date in which the sale was done
   - **price**: price of the sale
   - **sales**: number of sales.

## Merge Dataframes

In order to look better the data, we will need to merge the 3 dataframes into a single one.

In [160]:
df = pd.merge(df_geo, df_sales, on=["geoCluster"])
df = pd.merge(df_sku, df, on=["SKU"])
df

,SKU,Category,brandId,lagerUnitQuantity,trademark,countryOfOrigin,Group,geoCluster,cityId,ID,date,price,sales
0,24,Pomegranate,0,1.0,0,0,Tropical fruits,323,1,RR27958444,2021-03-03,79.49,0.4
1,24,Pomegranate,0,1.0,0,0,Tropical fruits,323,1,RR27958445,2021-03-04,NaN,NaN
2,24,Pomegranate,0,1.0,0,0,Tropical fruits,323,1,RR27958446,2021-03-05,NaN,NaN
3,24,Pomegranate,0,1.0,0,0,Tropical fruits,323,1,RR27958447,2021-03-06,NaN,NaN
4,24,Pomegranate,0,1.0,0,0,Tropical fruits,323,1,RR27958448,2021-03-07,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4605980,838137,Avocado,0,1.0,9666,0,Tropical fruits,3230,0,RR31319305,2021-07-01,39.79,7.0
4605981,838137,Avocado,0,1.0,9666,0,Tropical fruits,3230,0,RR31319306,2021-07-02,39.79,6.0
4605982,838137,Avocado,0,1.0,9666,0,Tropical fruits,3230,0,RR31319307,2021-07-03,39.79,21.0
4605983,838137,Avocado,0,1.0,9666,0,Tropical fruits,3230,0,RR31319308,2021-07-04,39.39,20.0


## Pandas Profiling

In [161]:
# from pandas_profiling import ProfileReport

In [162]:
# profile = ProfileReport(df, title="Pandas Profiling Report")
# profile

In [163]:
# profile.to_file("McKiensy.html")

## Refactor

In [164]:
df.date = pd.to_datetime(df.date) #Change data type to datetime
df = df.dropna(subset=['sales']).reset_index()
df.drop(['index', 'ID'], axis=1, inplace=True)

In [165]:
df

,SKU,Category,brandId,lagerUnitQuantity,trademark,countryOfOrigin,Group,geoCluster,cityId,date,price,sales
0,24,Pomegranate,0,1.0,0,0,Tropical fruits,323,1,2021-03-03,79.49,0.4
1,24,Pomegranate,0,1.0,0,0,Tropical fruits,1934,0,2020-11-05,46.89,1.4
2,24,Pomegranate,0,1.0,0,0,Tropical fruits,1934,0,2020-11-26,54.09,2.5
3,24,Pomegranate,0,1.0,0,0,Tropical fruits,1934,0,2020-11-28,54.09,1.4
4,24,Pomegranate,0,1.0,0,0,Tropical fruits,1934,0,2020-12-09,62.39,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
778361,838137,Avocado,0,1.0,9666,0,Tropical fruits,3230,0,2021-07-01,39.79,7.0
778362,838137,Avocado,0,1.0,9666,0,Tropical fruits,3230,0,2021-07-02,39.79,6.0
778363,838137,Avocado,0,1.0,9666,0,Tropical fruits,3230,0,2021-07-03,39.79,21.0
778364,838137,Avocado,0,1.0,9666,0,Tropical fruits,3230,0,2021-07-04,39.39,20.0


In [166]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778366 entries, 0 to 778365
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   SKU                778366 non-null  int64         
 1   Category           778366 non-null  category      
 2   brandId            778366 non-null  Int64         
 3   lagerUnitQuantity  778366 non-null  Float64       
 4   trademark          778366 non-null  Int64         
 5   countryOfOrigin    778366 non-null  Int64         
 6   Group              778366 non-null  category      
 7   geoCluster         778366 non-null  int64         
 8   cityId             778366 non-null  int64         
 9   date               778366 non-null  datetime64[ns]
 10  price              778366 non-null  float64       
 11  sales              778366 non-null  float64       
dtypes: Float64(1), Int64(3), category(2), datetime64[ns](1), float64(2), int64(3)
memory usage: 63.8 MB


In [167]:
# df['total_money'] = df['price'] * df['sales']
# df

## Train

In [76]:
df.Group.value_counts()

3    248117
0    217072
2    148953
4     92323
1     71901
Name: Group, dtype: int64

In [169]:
from sklearn.preprocessing import LabelEncoder

df['Category'] = LabelEncoder().fit_transform(df['Category'])
df['Group'] = LabelEncoder().fit_transform(df['Group'])

# df['date'] = df['date'].astype(np.int64)

In [175]:
import datetime as dt

df.date = pd.to_datetime(df.date) 
# df.date = df.date .map(dt.datetime.toordinal)

In [109]:
# Partició del conjunt de dades Housting en conjunt d'entrenament (60%) i conjunt de prova (40%)
from sklearn.model_selection import train_test_split

X = df.iloc[:, :-1].values
Y = df['sales'].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.8, random_state = 1)

In [178]:
df['brandId'] = df['brandId'].astype("int64")
df['Group'] = df['Group'].astype("int64")
df['Category'] = df['Category'].astype("int64")
df['SKU'] = df['SKU'].astype("int64")
df['geoCluster'] = df['geoCluster'].astype("int64")
df["cityId"] = df['cityId'].astype("int64")
df["date"] = df["date"].astype("float64")
df["price"] = df["price"].astype("float64")
df['lagerUnitQuantity'] = df['lagerUnitQuantity'].astype('float64')
df['trademark'] = df['trademark'].astype('int64')
df['countryOfOrigin'] = df['countryOfOrigin'].astype('int64')

In [194]:
copy_df = df.copy()

In [195]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778366 entries, 0 to 778365
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   SKU                778366 non-null  int64  
 1   Category           778366 non-null  int64  
 2   brandId            778366 non-null  int64  
 3   lagerUnitQuantity  778366 non-null  float64
 4   trademark          778366 non-null  int64  
 5   countryOfOrigin    778366 non-null  int64  
 6   Group              778366 non-null  int64  
 7   geoCluster         778366 non-null  int64  
 8   cityId             778366 non-null  int64  
 9   date               778366 non-null  float64
 10  price              778366 non-null  float64
 11  sales              778366 non-null  float64
dtypes: float64(4), int64(8)
memory usage: 71.3 MB


In [42]:
# Entrenem i evaluem un model de regressió multivariable per bosc aleatori
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Inicialitzem i entrenem un model de regressió per bosc aleatori amb criteri d'impuresa 'MSE'
forest = RandomForestRegressor(n_estimators = 1000, criterion = 'mse', random_state = 1, n_jobs = -1)
forest.fit(X_train, Y_train)

# Resultat de predicció
Y_train_pred = forest.predict(X_train)
Y_test_pred = forest.predict(X_test)

# Evaluacions de rendiment
print('MSE train: %.3f test %.3f' % (mean_squared_error(Y_train, Y_train_pred), mean_squared_error(Y_test, Y_test_pred)))
print('R^2 train: %.3f test %.3f' % (r2_score(Y_train, Y_train_pred), r2_score(Y_test, Y_test_pred)))

MSE train: 1.715 test 13.677
R^2 train: 0.918 test 0.365


In [ ]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

# Model de regressió lineal bàsic
regr = LinearRegression()

# Tranformació del conjunt X en X_quad i X_cub
quadratic = PolynomialFeatures(degree = 2)
cubic = PolynomialFeatures(degree = 3)
X_quad = quadratic.fit_transform(X)
X_cubic = cubic.fit_transform(X)

# Rang de valors homogeni pels resultats gràfics dels models
X_fit = np.arange(X.min(), X.max(), 1)[:, np.newaxis]

# Ajustament dels models lineal, quadràtic i cúbic
regr = regr.fit(X, Y)
Y_lin_fit = regr.predict(X)
linear_r2 = r2_score(Y, regr.predict(X))

regr = regr.fit(X_quad, Y)
Y_quad_fit = regr.predict(quadratic.fit_transform(X))
quadratic_r2 = r2_score(Y, regr.predict(X_quad))

regr = regr.fit(X_cubic, Y)
Y_cubic_fit = regr.predict(cubic.fit_transform(X))
cubic_r2 = r2_score(Y, regr.predict(X_cubic))

# Grafiquem resultats
plt.figure(figsize=(10, 6))

plt.scatter(X, Y, label = 'Training points',color = 'lightblue', edgecolor = 'white')

plt.plot(X_fit, Y_lin_fit, label = 'linear (d = 1), $R^2=%.3f$' %linear_r2, 
         color = 'blue', lw = 2, linestyle = ':')

plt.plot(X_fit, Y_quad_fit, label = 'quadratic (d = 2), $R^2=%.3f$' %quadratic_r2, 
         color = 'red', lw = 2, linestyle = '-')

plt.plot(X_fit, Y_cubic_fit, label = 'cubic (d = 3), $R^2=%.3f$' %cubic_r2, 
         color = 'green', lw = 2, linestyle = '--')

plt.xlabel('% lower status of the population [LSTAT]')
plt.ylabel('Price in $1000s [MEDV]')
plt.legend(loc = 'upper right')
plt.show()

## TensorFlow Modelling

In [196]:
t = df.copy()

In [181]:
from sklearn.model_selection import train_test_split

def get_data_model(df, test_size):
    target = df.pop('sales')
    X_train, X_test, Y_train, Y_test = train_test_split(df,target, test_size=test_size)
    return X_train, X_test, Y_train, Y_test

X_train, X_test, Y_train, Y_test = get_data_model(t, 0.2)

In [63]:
numeric_features_train = tf.convert_to_tensor(X_train)
numeric_features_test = tf.convert_to_tensor(X_test)

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features_train)
normalizer.adapt(numeric_features_test)

In [72]:
def get_basic_model():
    model = tf.keras.Sequential([
        normalizer,
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(1)
      ])

    model.compile(optimizer='adam',
                loss=tf.keras.losses.MeanSquaredError(),
                metrics=['RootMeanSquaredError'])
    return model

model = get_basic_model()
history = model.fit(numeric_features_train, Y_train, epochs=10, validation_data=(numeric_features_test, Y_test))

Epoch 1/10
19460/19460 [==============================] - 29s 1ms/step - loss: 16.3981 - root_mean_squared_error: 4.0495 - val_loss: 19.3326 - val_root_mean_squared_error: 4.3969
Epoch 2/10
19460/19460 [==============================] - 29s 2ms/step - loss: 15.9276 - root_mean_squared_error: 3.9909 - val_loss: 19.1631 - val_root_mean_squared_error: 4.3776
Epoch 3/10
19460/19460 [==============================] - 29s 2ms/step - loss: 15.7316 - root_mean_squared_error: 3.9663 - val_loss: 19.2196 - val_root_mean_squared_error: 4.3840
Epoch 4/10
19460/19460 [==============================] - 29s 2ms/step - loss: 15.6210 - root_mean_squared_error: 3.9523 - val_loss: 18.9875 - val_root_mean_squared_error: 4.3575
Epoch 5/10
19460/19460 [==============================] - 29s 2ms/step - loss: 15.5496 - root_mean_squared_error: 3.9433 - val_loss: 18.7726 - val_root_mean_squared_error: 4.3327
Epoch 6/10
19460/19460 [==============================] - 30s 2ms/step - loss: 15.4942 - root_mean_square

## Training with Groups

In [197]:
df_group3 = t[t.Group == 3]

In [198]:
X_train, X_test, Y_train, Y_test = get_data_model(df_group3, 0.2)

In [199]:
numeric_features_train = tf.convert_to_tensor(X_train)
numeric_features_test = tf.convert_to_tensor(X_test)

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features_train)
normalizer.adapt(numeric_features_test)

In [119]:
model = get_basic_model()
history = model.fit(numeric_features_train, Y_train, epochs=10, validation_data=(numeric_features_test, Y_test))

Epoch 1/10
6203/6203 [==============================] - 10s 1ms/step - loss: 20.1928 - root_mean_squared_error: 4.4936 - val_loss: 16.4930 - val_root_mean_squared_error: 4.0612
Epoch 2/10
6203/6203 [==============================] - 9s 1ms/step - loss: 19.7119 - root_mean_squared_error: 4.4398 - val_loss: 16.5647 - val_root_mean_squared_error: 4.0700
Epoch 3/10
6203/6203 [==============================] - 9s 1ms/step - loss: 19.4979 - root_mean_squared_error: 4.4156 - val_loss: 16.0282 - val_root_mean_squared_error: 4.0035
Epoch 4/10
6203/6203 [==============================] - 9s 2ms/step - loss: 19.2882 - root_mean_squared_error: 4.3918 - val_loss: 15.8647 - val_root_mean_squared_error: 3.9830
Epoch 5/10
6203/6203 [==============================] - 9s 1ms/step - loss: 19.0670 - root_mean_squared_error: 4.3666 - val_loss: 16.0752 - val_root_mean_squared_error: 4.0094
Epoch 6/10
6203/6203 [==============================] - 9s 2ms/step - loss: 18.9656 - root_mean_squared_error: 4.3549 -

## SW Modelling per Category

In [208]:
import copy

class ModellForecastCategory:
    def __init__(self, model_gen, num_categories):
        self.models = [model_gen() for x in range(num_categories)]
        self.num_categories = num_categories
        self.historials = {}
        
    def fit_group(self, X_train, y_train, X_test, y_test, num_category):
        self.historials[num_category] = self.models[num_category].fit(X_train, 
                                                                      y_train, 
                                                                      epochs=30, 
                                                                      validation_data=(X_test, Y_test))
        
    def predcit(self, X, num_category):
        return self.models[num_category].predict(X)

In [209]:
macroModel = ModellForecastCategory(get_basic_model, 5)

In [210]:
testing_df[testing_df.Group == 4]

,SKU,Category,brandId,lagerUnitQuantity,trademark,countryOfOrigin,Group,geoCluster,cityId,date,price,sales
320745,89074,13,2724,28.0,1835,0,4,131,1,1.623110e+18,20.79,2.0
320746,89074,13,2724,28.0,1835,0,4,491,1,1.624493e+18,21.39,4.0
320747,89074,13,2724,28.0,1835,0,4,683,1,1.621210e+18,18.09,4.0
320748,89074,13,2724,28.0,1835,0,4,1321,0,1.618186e+18,20.79,7.0
320749,89074,13,2724,28.0,1835,0,4,1321,0,1.623197e+18,20.79,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
771787,819150,39,7358,350.0,6133,0,4,3209,0,1.625357e+18,56.89,6.0
771788,819150,39,7358,350.0,6133,0,4,3230,0,1.625098e+18,56.89,4.0
771789,819150,39,7358,350.0,6133,0,4,3230,0,1.625270e+18,56.89,6.0
771790,819150,39,7358,350.0,6133,0,4,3230,0,1.625357e+18,56.89,14.0


In [211]:
testing_df = df.copy()

In [ ]:
for group in testing_df.Group.unique():
    group_data = testing_df[testing_df.Group == group].copy()
    X_train, X_test, Y_train, Y_test = get_data_model(group_data, 0.2)
    
    numeric_features_train = tf.convert_to_tensor(X_train)
    numeric_features_test = tf.convert_to_tensor(X_test)

    normalizer = tf.keras.layers.Normalization(axis=-1)
    normalizer.adapt(numeric_features_train)
    normalizer.adapt(numeric_features_test)
    
    print('-------------------------')
    print('Training number: ',group)
    macroModel.fit_group(numeric_features_train, Y_train, numeric_features_test, Y_test, group)
    print('-------------------------')

-------------------------
Training number:  3
Epoch 1/30
6203/6203 [==============================] - 10s 2ms/step - loss: 20.6052 - root_mean_squared_error: 4.5393 - val_loss: 14.9060 - val_root_mean_squared_error: 3.8608
Epoch 2/30
6203/6203 [==============================] - 9s 2ms/step - loss: 20.1035 - root_mean_squared_error: 4.4837 - val_loss: 14.7515 - val_root_mean_squared_error: 3.8408
Epoch 3/30
6203/6203 [==============================] - 9s 2ms/step - loss: 19.9022 - root_mean_squared_error: 4.4612 - val_loss: 14.7909 - val_root_mean_squared_error: 3.8459
Epoch 4/30
6203/6203 [==============================] - 9s 2ms/step - loss: 19.7076 - root_mean_squared_error: 4.4393 - val_loss: 14.5244 - val_root_mean_squared_error: 3.8111
Epoch 5/30
6203/6203 [==============================] - 10s 2ms/step - loss: 19.5334 - root_mean_squared_error: 4.4197 - val_loss: 14.3700 - val_root_mean_squared_error: 3.7908
Epoch 6/30
6203/6203 [==============================] - 10s 2ms/step - l

## Test

In [10]:
test

,ID,geoCluster,SKU,date,price_filled,sales
0,RR27956447,21,32485,2021-07-08,39.69,NaN
1,RR27956448,21,32485,2021-07-09,39.69,NaN
2,RR27956449,21,32485,2021-07-10,39.69,NaN
3,RR27956450,21,32485,2021-07-11,39.69,NaN
4,RR27956451,21,32485,2021-07-12,39.69,NaN
...,...,...,...,...,...,...
202732,RR55912539,3230,711838,2021-07-15,28.49,NaN
202733,RR55912540,3230,711838,2021-07-16,28.49,NaN
202734,RR55912541,3230,711838,2021-07-17,28.49,NaN
202735,RR55912542,3230,711838,2021-07-18,28.49,NaN


In [41]:
test = pd.merge(df_geo, test, on=['geoCluster'])

In [201]:
test.date.value_counts()

2021-07-19    14589
2021-07-18    14570
2021-07-17    14553
2021-07-16    14539
2021-07-15    14515
2021-07-14    14494
2021-07-13    14489
2021-07-12    14476
2021-07-11    14463
2021-07-10    14445
2021-07-09    14430
2021-07-08    14411
2021-07-07    14389
2021-07-06    14374
Name: date, dtype: int64